### 目录
---
- #### 何为风险模型
    - 从阿波罗登月说起
    - 收益的来源
    - 理解模型
    - 搭建模型
- #### 为什么要使用风险模型
    - 从完整的投资流程说起
    - 盘前
    - 盘中
    - 盘后
    - 总结和反思
- #### 值得深入思考的几个问题

### 1、什么是风险模型
--- 
- 有句话说得好，只有痛过之后才会记忆深刻，在A股市场的表现更是淋漓尽致
- 从2007年至今，SIZE因子基本上是打遍天下无敌手，两次比较大的痛就是14年12月和17年4月，由于14年处于牛市，有足够的盈余可以扛住回撤，但今年不一样，大部分打着“量化”名头的产品都是亏的，一时间市场对量化的各种非议漫天遍野，我还听到过一个段子
 	- A：请问你是做什么的？
 	- B：搞量化的
 	- A：喔，撸中小创的
- 作为一个严肃认真的量化实践者我有必要为量化正名，于是乎便写了一篇文章[`基于长信量化先锋的归因分析`](https://uqer.io/community/share/5924264f1bfe1200587f2f07)，试图从客观公正、带有正统量化精神和情怀的角度来解答市场对量化的偏见和误解
- 文章发布便受到了各路同行的关注，尤其是公募的朋友，因为分析是基于我们自有的风险模型，所以在过去一段时间里我便有了很多和同行深入交流风险模型的机会
- 但在交流过程中我发现了很多问题，国内的量化从业者大多对风险模型了解不深，于是乎就想着写一篇文章，系统的从介绍下风险模型的前世和今生，所不同的是本文完全从买方或者说实战角度来审视风险模型，所以关于很多繁琐的公式推导、烂大街的套话这里就不详细展开，这块有兴趣的可以看看各大券商关于风险模型的介绍文章，又或者是追本溯源，直接去看barra USE3、USE4等

#### 1.1、从阿波罗登月说起
---
- 1969年7月16日，美国宇航局发射阿波罗11号飞船成功登录月球，自此人类实现了登月的梦想，在感慨人类伟大创造力的同时不禁会想是什么创造了这一切
- 没错，是人类的智慧，人类之所以能主宰地球，也主要是因为人类强大的归纳和推导能力，用哲学的话说应该是发现规律、利用规律
- 其实，从我们踏入学堂开始，我们就在接受先辈们给我们发现的一套理论体系，苹果从树上掉下来我们便想起了牛顿的万有引力，进而我们利用万有引力实现了人类的飞天梦，那么引力实际上是什么呢？磁场又是个什么东西呢？我们实际上感知不到，甚至我们可以假设有比人类更高阶的一个物种可以用xxx理论来解释自然现象，从而打败我们从小习知所有知识，但就目前来看，在已知的所有理论体系下，我们解释了自然现象，同时也将其用到我们的生活中
- 那么同样的，对于股市我们的思路也一样，`我们试图寻找一个理论体系来解释股票的收益，从而将其用在我们的投资中`，而风险模型就是其中的万有引力定律

#### 1.2、收益的来源
---
- 承接上文，我们的目的是构建一套理论体系用以解释股票的收益，最一般的来讲，就是站在今日24:00解释明天股市所有股票的日收益率
- 那么我们如何去解释呢？首先要发现规律，我们发现每天同一个行业的股票涨跌幅比较接近，又发现市值大股票和市值小股票走势有明显差异，那么一个直觉是我们就可以用行业和市值这两个属性来作为我们解释次日股票收益的工具
- 更进一步，我们从公司财务、市场行为、行为金融等角度来寻找能够解释次日收益来源的属性，那么就诞生了风险模型的其实变量：`因子暴露Exposure`，这里面又可以细分为行业因子和风格因子，行业因子不用解释，上面提到的市值因子就属于风格因子，其他的风格因子还有估值、成长、盈利质量等
- 假设我们现在已经在24:00获取了所有股票的因子暴露，那么就开始解释明日股票收益，方法就是大伙熟知的多元线性回归，下面将详细介绍

#### 1.3、理解模型
---
- 多元线性回归的方程是$\boldsymbol r = \boldsymbol X \boldsymbol f + \boldsymbol u$，其中$\boldsymbol r$为n*1列向量，代表着n个股票的次日收益率，$\boldsymbol X$为上面提到的因子暴露Exposure，$\boldsymbol f$为因子收益factor return，$\boldsymbol u$为股票特有收益specific return
- 这样我们就实现了对次日股票收益的解释，又或者说，我们将股票的收益可以拆分成公共部分和特有部分，公共部分就是Exposure的部分，比如市值因子带来的收益是多少，股票特有的收益是多少
- 听到这里，大伙肯定很好奇，就这么一个Exposure就可以解释明天股票的收益了吗？当然不是全部，每天的回归都会产生一个rsquare，我们统计了滚动40天的rsquare，平均值接近50%，最高时接近90%，最低时也不低于20%，这也就是说我们的模型只解释了收益的50%，应该算是非常高的解释力，讲个笑话，要是再高些的话就没有pure alpha可以搞了。。
- 那么是不是我们每天做这么个回归就好了呢？当然不是的，一般的介绍风险模型都是从预测组合风险入手，那么我们就需要预测一个包括所有股票的方差协方差矩阵，但是由于历史数据不够、需要估计的变量太多等因素，所以就将估计问题转到预测公共因子和特质因子的波动上，这类介绍是市面上的通用术语，但我们这里从更深的层次引入风险模型的概念，从对收益的解释入手，这样才好理解模型本身的诸多应用

#### 1.4、模型搭建
---
- 在做完每天的回归之后，我们便有了每天的factor return数据和specific return数据，这样我们就可以估计公共因子的方差协方差矩阵$\boldsymbol F$，股票的特质波动率$\boldsymbol \Delta$
- 于是乎，我们便有了所有股票的方差协方差矩阵$\boldsymbol V$，基于此我们便可以计算任何股票组合的预测风险值，假设组合持仓为$\boldsymbol h$，那么组合波动率可以表示为$\boldsymbol h' (\boldsymbol X \boldsymbol F \boldsymbol X' + \boldsymbol \Delta) \boldsymbol h$
- 至此，风险模型的几大核心部件都已经介绍完了，分别是因子暴露Exposure、因子收益factor return、特有收益specific return、因子协方差矩阵factor covaraicne、股票特有风险specific risk
- 至此我们对风险模型的理念和整体框架做了大致的介绍，很多算法细节大伙有兴趣可以详细阅读barra相关文档，这一块不是本文重点；好了，我们要开始重点内容了，前戏已经够多了，来来来，不要停

### 2、为什么要使用风险模型
---
- 在过去的两个月里，我有幸和众多专业机构就风险模型进行了深入交流，但发现的结果是大家对风险模型了解甚少，有在使用的机构也没有发挥到模型的最佳价值
- 为了让本部分更有条理性以及更有说服力，我们将从实战的角度来阐述使用风险模型的理由，从盘前、盘中、盘后等方方面面来介绍风险模型的使用价值

#### 2.1、从完整的投资流程说起
--- 
- 如下图所示，一个完整的投资流程应该包含如下几个步骤，简单来看就是盘前、盘中和盘后
- **盘前**
	- 首先是更新信号库，那么在信号的研究和计算的过程中我们是需要风险模型进行`信号提纯`
	- 然后是策略生成，那么我们需要用到风险模型数据进行`组合构建`，生成满足我们对风险和收益要求的持仓
	- 最后是`风格监控`，市场行情瞬息万变，我们需要动态了解市场微观结构的潜在变动，然后最好相应的对策
- **盘中**：这一块主要涉及监控和交易，和盘前和盘后内容重复，不详细介绍
- **盘后**
	- `收益归因`：一天行情结束了，不管我们组合是赢是亏，我们总要知道发生了什么，是什么给我带了的收益，正如我们开篇提到的风险模型解释了50%的收益，那么我们组合的收益在50%的解释里主要集中在哪些风险因子上，pure alpha收益又有多少
	- `风险归因`：同样，在风险模型的理论框架下，我们不仅能将收益拆解，风险同样可以拆解，让我们可以看得更仔细
	- `因子暴露/组合偏离分析`：组合在经过几天行情之后，必然会和我们当初建仓时理想的状态有所偏离，比如涨幅多的行业可能会出现高配，同时在风格上也会产生些许偏差，那么就需要对组合的最新持仓进行偏离度分析，做到动态了解，知根知底

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/2c5d752c-6d3d-11e7-bc8b-0242ac140002)

#### 2.2、盘前
--- 
- ##### A、信号提纯
	- 可能很多人对信号提纯不太了解，但我们在做因子研究的时候有一项测试是肯定要做的，就是看测试因子和常见因子的相关性，若高度相关则表明因子价值不大，那么信号提纯就是干这事的
	- 简单讲就是`用测试因子对常见风格因子和行业因子进行线性回归，取其残差项`，那么这样就保障了信号本身不和常见的风格因子相关，提取出来的就是纯的alpha
	- 在优矿上我们提供的neutralize()函数就是干这事的，你们可以在帮助文档可以看到该函数的详细用法
	- 在如上的这个步骤中，我们就需要用到风险模型的**Exposure数据**，正如前面介绍，exposure就是市场上最最常见的风格因子和行业因子构成的矩阵，刚好拿来直接用，在优矿中我们可以非常方便的调用如下DataAPI获取

In [ ]:
DataAPI.RMExposureDayGet(tradeDate="20170719").head()

,tradeDate,ticker,secID,exchangeCD,secShortName,BETA,MOMENTUM,SIZE,EARNYILD,RESVOL,...,AgriForest,CHEM,Media,IronSteel,NonBankFinan,ELECEQP,AERODEF,Conglomerates,COUNTRY,updateTime
0,20170719,000001,000001.XSHE,XSHE,平安银行,-1.052,-0.115,1.764,0.962,-0.833,...,0,0,0,0,0,0,0,0,1,2017-07-19 23:48:39
1,20170719,000002,000002.XSHE,XSHE,万科A,0.821,0.542,1.764,1.736,0.914,...,0,0,0,0,0,0,0,0,1,2017-07-19 23:48:39
2,20170719,000004,000004.XSHE,XSHE,国农科技,0.047,-1.808,-3.118,0.043,1.224,...,0,0,0,0,0,0,0,0,1,2017-07-19 23:48:39
3,20170719,000005,000005.XSHE,XSHE,世纪星源,1.089,-1.494,-1.937,-0.357,-0.355,...,0,0,0,0,0,0,0,0,1,2017-07-19 23:48:39
4,20170719,000006,000006.XSHE,XSHE,深振业A,0.739,-0.075,-1.076,1.636,0.270,...,0,0,0,0,0,0,0,0,1,2017-07-19 23:48:39


##### B、组合构建
---
- 在我们更新好每天的信号数据之后，就要开始构建最新的理想组合，在确定好风格、行业偏好、相关参数之后，就可以构建出组合
- 简单讲，组合构建就是求解一个带限制条件的二次优化问题，一个典型的例子如下（其他很多种变形这里不详细讨论）
$$max \quad \boldsymbol w' \boldsymbol \alpha - \lambda \boldsymbol w' \boldsymbol \Omega \boldsymbol w$$
$$ s.t. \quad industry \quad constrains$$
$$ \quad style \quad constrains$$
$$ other \quad constains$$
$$ \boldsymbol w' \boldsymbol 1 = 1$$
$$ \boldsymbol w\geq0$$
- 上式中的风险预测矩阵$\boldsymbol \Omega$就是来自风险模型，$\boldsymbol \Omega = \boldsymbol X \boldsymbol F \boldsymbol X' + \boldsymbol \Delta$
- 另外，在各种constrains中也需要用到Exposure矩阵
---
- 从上面可以看到，`组合构建就是提供了一套完整的理论和实践框架，让我们对组合的方方面面做好把控，什么风险值得去主动暴露、什么风险又必须控制好`
- 说到这里就特别想提下今年独特的市场行情，若没有这次的小盘股风波，恐怕国内众多的量化产品依然会无视对风险的把控，依然在迷糊中量化
- 另外还想提一点，国内众多卖方金工在做因子研究构建组合时，就是简单的行业中性下等权处理，由于国内各大指数基本是市值加权的，所以等权构建组合必然会带来小市值暴露，但这一点卖方金工是不会告诉你的（控制市值后因子效果会有一定折扣，这样影响阅读量的事是肯定不会干的）
- 所以，我们看到今年的量化产品出现了两极分化，没控制市值风险的就排名垫底，控制了市值风险的就屡创新高，未来IPO还会继续，愉快搞小票的时代一去不复返了，未来估计要看真正的量化本事了
- 作为一个严肃认真的量化实践者，我在优矿上的参赛策略也在近期屡创新高，15年10月至今绝对收益高达38%，累计跑赢沪深300指数26%，信息比高达3.06，贴图供大家娱乐娱乐（**`我们的态度：优矿做量化是认真的！`**）

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/95af2042-6d48-11e7-bc8b-0242ac140002)

##### C、风格监控
---
- 如果说投资是一个军队参加战争的话，那么对风格的监控绝对是冲在一线的精锐部队，或者叫特种部队
- 正如前面介绍，每天早上我们在构建组合时需要确定风格、行业偏好，那么如何确定呢？一个很自然的想法就是看看近期市场是什么状况，看看各风格因子长期、短期表现如何，市场结构是否发生了变化
- 风险模型中的一个中间产物叫做因子收益factor return，应该都还有印象吧，简单来讲factor return可以近似作为该类风格的代表（PS：每一个factor return都对应着一个factor portfolio，这个factor portfolio可以近似代表这该风格组合的表现，当然更精确的做法是构建style unit alpha portfolio，这一块不在这里细说，因为这两者差异不大，整体趋势是一致的）
- 以SIZE因子为例进行说明，我们可以通过DataAPI.RMFactorRetDayGet获取到factor return的历史数据，这里直接给结果

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/cac8ce6a-6d4b-11e7-bc8b-0242ac140002)

#### 对上图说明几点
---
- 上面两张图分别绘制的是SIZE factor cumulative return的长期和短期走势
- 左边这张图很明显看到累计收益稳定为负，代表了在A股市场上极其明显的小市值效应，有两个地方值得注意下，一个是14年12月，一个是17年4月以来，可以看到累计收益掉头向上，代表着明显的小盘股之殇
- 右边这张图是最近一年SIZE因子的表现，可以看到从16年10月以来风格就开始有了微妙的变化，倘若我们能够找找发现，再结合政策分析，应该是能够避免掉这一波的，但不管怎么说，我们可以利用好这个监控雷达，避免雷区&发现战机
- 更直观的，我们可以通过如下一张表格来看看10大风格因子的表现，每天早上动态监控

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/f7080036-6d4b-11e7-bc8b-0242ac140002)

#### 特殊福利
---
- 看到这里大伙应该都有点冲动想看看其他风格因子的走势吧，看看有没有值得bet的，你们想得到我当然也想得到啦
- 下图是盈利质量因子EARNYILD的走势图，我不会告诉你们，我最近靠这个因子跑赢了指数不少的，我也不会告诉你，我上面展示的策略在近期可是对EARNYILD有刻意的主动暴露的，低调低调
- 严肃点，咋们是搞投资呢。。我是这么看这个问题的
	- `IPO快马加鞭、供给侧改革去产能，那么在现有经济持续探底的大背景下，大企业相比小企业会呈现剩着为王的局面，同时各大央企的重组合并，未来不乏会出现新的垄断形式，规模优势明显`
	-  `MSCI纳入A股、房价暂时稳住，带来的资金自然会找出处，在美国加息的背景下国内债市不会有太多机会，那么股市作为替代品必然会部分受益，而这些大规模资金天生是风险厌恶者，对比上面一条的分析，必然会选择估值低、未来有稳定业绩预期的行业龙头（当然又是龙头又有较快增长的更好，比如消费升级下的白色家电和白酒）`
	-  `简单讲未来会是一个对盈利质量高度重视的市场，我不敢断言现在的蓝筹股行情走了多久，未来或许有回调，但回调过后我还是更愿意选择白马蓝筹，业绩在那我更有信心`
	-  `yy一下，长期来看，也行现在的蓝筹行情上半场才刚结束，下半场或许更精彩`

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/000610d8-6d4c-11e7-bc8b-0242ac140002)

#### 2.3、盘中
---
- 盘中这块涉及更多的是交易以及一些监控，比如大类信号表现的实时监控、组合alpha实时监控等
- 由于这部分内容大多和盘后的归因一致，只是多了实时性的要求，这里就不详细展开
- 另外，个人是非常提倡在多因子的基础上再加上些能增加收益的其他策略，比如T+0，那么同样的，我们可以将组合收益进行更加细致的分解，首先分解到各个小策略层面（择时/T+0等），然后在分解到各个因子上，不管怎么说，这一套理论体系受用到我们投资的每个地方

#### 2.4、盘后
--- 
- ##### A、收益归因
	- 这一块的内容在之前的[`长信量化归因分析`](https://uqer.io/community/share/5924264f1bfe1200587f2f07)一文中有过详细的介绍，简单来说，我们可以根据前面介绍的每天回归等式$\boldsymbol r = \boldsymbol X \boldsymbol f + \boldsymbol u$，将组合收益分解成为风格因子收益 + 行业因子收益 + 特有收益（alpha收益）
	- 下面以我自己实际组合的某一天情况为例进行说明，2017年7月18日组合相比沪深300有24bps主动收益，那么这24bps是否真的是alpha收益呢，还是只是说某个风格带来的收益，上面我也提到了说我刻意对EARNYILD有主动暴露（因为我就是觉得EARNYILD要很牛逼），那么这一块是否给我带了收益呢，下图将详细分解

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/e576b57a-6dc2-11e7-bc8b-0242ac140002)

##### 对上图的说明
---
- 为了说明的更清楚，图中绘制了组合对各风格的主动暴露情况以及对应在风格上的主动收益，最右边的SRET是特有收益，也就是alpha收益，所以是没有主动暴露的
- 从图中很清楚看到，由于组合对EARNYILD有0.65的主动暴露，由此带来了32bps的主动收益，而对BTOP的主动暴露带来了-10bps主动收益，最后，可以看到，我们的alpha收益大概是7bps
- 当然，我们可以更进一步的分析，看看这7bps的alpha是由哪些因子带来的，这个就需要用到构建多因子时的各构成因子的值，然后进行分解，不过我个人觉得再次分解价值不大，因为在研究因子的时候我肯定是清楚单个因子大概都是偏好什么风格的，哪几个确确实实的是和大类风格因子不相关的alpha因子
- 同样的，我们也对行业因子绘制了如上的分解图，如下所示，由于我的组合做了行业中性处理，所以这一块没什么大文章，没有完全和benchmark对齐主要是因为行情自身的涨跌以及A股整手买卖的限制，这一点和后面要介绍的组合偏离度分析一致
- 总的来说，`通过分解，我们可以将组合24bps的主动收益分解到风险模型的框架上`，一分不多一分不少，刚刚好

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/f14a8d5e-6dc2-11e7-bd38-0242ac140002)

##### B、风险归因
---
- 有收益就有风险，我们虽然取得了24bps的主动收益，但是我们要看看收益的来源是否是我们主动承担的风险，比如上面的EARNYILD，有主动暴露能够带来主动收益就是好事，但是像BETA这种，有主动暴露但是带来了负收益，那么这个风险就不要去承担
- 和收益归因类似，我们也可以将组合风险拆解到各个因子上，然后看看主动风险的具体来源是哪？一个很直观的感觉是，主动风险必然和主动暴露高度相关，事实上，可以将主动风险定义如下
$$active \quad risk = x \sigma \rho$$
- 也就是说，主动风险是和主动因子暴露$x$、因子自身波动率$\sigma$、因子和组合的相关性$\rho$，来共同决定的
- 在实践中，我个人更倾向于用主动暴露来代替主动风险，一方面是更直观，另一方面是因为我们在组合构建里控制的就是主动暴露，倘若主动暴露为0，那么主动风险必然为0
- 而主动暴露的计算是每天都要干的一件事，下面就详细看看组合偏离分析

##### C、因子暴露/组合偏离分析
---
- 如之前描述，组合在经过几天行情之后，必然会和我们当初建仓时理想的状态有所偏离，比如涨幅多的行业可能会出现高配，同时在风格上也会产生些许偏差，那么就需要对组合的最新持仓进行偏离度分析，做到动态了解，知根知底
- 所以在每天盘后我们都需要计算组合在风格、行业上和基准的偏差，看看是否和理想的目标值偏差太大，若偏离太大可能就有必要介入微调或是调仓
- 如下图，我就计算了组合相比基准和行业的主动暴露情况，为了对比方便，我把基准在这些因子上的暴露也列示出来，这样就可以看得更清楚

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/fa122892-6dd7-11e7-bd38-0242ac140002)

![图片注释](http://storage.uqer.io/548e9868f9f06c45e7073e66/04904934-6dd8-11e7-bc8b-0242ac140002)

##### 然后呢？
---
- 一天行情结束了，我们就再来一个轮回，继续盘前——盘中——盘后的步骤
- 我们可能还需要不断研究新的alpha因子，我们也还需要监控近期市场表现，看看是否发生了大的变化
- 这一切都是周而复始的轮回，而我们的风险模型理论体系贯穿在每个角落，就好比生活中的任何角落都离不开数学一样

#### 2.5、总结和反思
---
- 好了，到现在我们从一个完整的流程介绍完了风险模型的价值，不知道各位看官是否找到了使用风险模型的理由
- 回到我们介绍风险模型的切入点：用一套理论体系解释股票收益的来源，而不仅仅是预测风险，这一点就是最好的诠释

### 3、值得深入思考的几个问题
---
- 这一部分本来想详细展开的，内容也主要是来源于近期和专业机构投资者交流下来的研究思路或者成果，但看到本文篇幅已经够多了，所以这里就抛砖引玉下，列个提纲，后续有时间再详细交流
- `day/short/long`：风险模型不就是预测风险吗？搞个day/short/long三个版本是什么鬼？
- `factor portfolio vs factor return`：如何深入理解风险模型里的factor return，每一个factor return其实都对应着一个factor portfolio，组合长啥样呢？
- `sizenl factor return之谜`：如果用过风险模型的都是知道sizenl的累积factor return是异常稳定的，但如何获取这部分收益呢？
- `如何理解sizenl`：在SIZE因子基础上怎么又来了个sizenl，如何理解？做多中盘股同时做空大小盘股？为什么其他因子exposure分布都是正太的，而sizenl分布却这么非主流呢？
- `不同universe风险模型`：针对A股就一个风险模型就够用了吗？那大盘股和小盘股对某些因子表现确实不太一样又该怎么搞？
- 欢迎补充。。。
---
#### **`最后，欢迎认可或者对我们模型感兴趣的金主爸爸来开通试用，也欢迎直接购买，养家糊口不容易，全指望年底奖金了，捂脸。。。`**